## 简介

迁移学习主要是将预训练模型（通常在ImageNet等大型高难度数据集上训练过）的一部分作为起点，通过一些改动获得对新项目有用的信息。迁移学习主要分为两种：1）特征提取（feature extraction）和2）微调（fine-tuning）。

### 特征提取（Feature Extraction）

<div>
<img src="https://pyimagesearch.com/wp-content/uploads/2019/05/transfer_learning_keras_feature_extract.png" width="400"/>
</div>

特征提取主要是将预训练的卷积神经网络作为一个任意的特征提取器。在上图中，我们可以看到，左边的是经典的VGG16网络架构，输出的是1000个ImageNet中的类别（class）的概率。右边的是去掉全连接层的VGG16网络，输出的是最后一个池化层。这时，我们就可以将输出作为我们提取的特征。

事实上，不仅是在最后的全连接层，我们可以在任意的一层停止正向传播，从而获取那一层输出的值作为我们提取的特征矢量（feature vector）。如果只去掉最后的全连接层，最后一层将变成最大池化层，输出形状为7\*7\*512的特征矢量。当我们或者这些特征矢量之后，我们就可以将他们输入进支持向量机，逻辑回归等机器学习模型中，从而获得一个新的分类器来区分目标物体类别。

### 微调（fine-tuning）

微调主要是将预训练模型顶端原有的一个或多个全连接层去掉，替换成新的一个或多个全连接层并对这些权重进行*微调*，同时在原有模型和全连接层之间也可以加入新的卷积层等常用层。

当使用特征提取的时候，我们并没有重新训练卷积神经网络，而是把它简单的当做一个特征提取器。而微调不仅需要我们更新网络的结构，还需要我们重新训练它去学习新的类别。值得注意的是，在微调时时非常容易产生过拟合的情况。

微调一般包含以下步骤：
* 移除全连接层并初始化新的全连接层
* 冻结网络中之前的卷积层，从而保护之前在预训练中习得的特征不被损坏
* 仅训练全连接层
* 可依据情况选择解冻部分卷积层进行第二次训练

在初始化新的全连接层后训练网络时，一般使用较低的学习率（learning rate），从而使新的全连接层能学习到之前的卷积层中的图案（pattern），这个过程叫做给全连接层”热身“。通过微调的迁移学习，我们通常会获得比通过特征提取的迁移学习更高的准确率。

在下图中，我们可以看到，左边的是VGG16的网络架构。中间是之前提到的，通过去掉全连接层来使用最后池化层的输出作为特征提取器。而右边的是微调的操作，通过初始化新的全连接层来替换掉之前的全连接层，从而在训练之后可以输出与训练集相关的类别。

<div>
<img src="https://pyimagesearch.com/wp-content/uploads/2019/06/fine_tuning_keras_network_surgery.png" width="600"/>
</div>

下面的这张图中展示了冻结网络中之前的卷积层。左边的网络架构显示，在我们开始微调过程时，我们先将所有的卷积层冻结，只允许梯度通过新的全连接层反向传播。右边的网络架构显示，在全连接层”热身“结束之后，我们可以解冻全部（或部分）卷积层来对每一（部分）层进行微调。

<div>
<img src="https://pyimagesearch.com/wp-content/uploads/2019/06/fine_tuning_keras_freeze_unfreeze.png" width="600"/>
</div>

在某些案例中，我们不需要解冻之前的卷积层就可以获得足够的准确率。但在大多数情况下，我们都需要解冻卷积层来获得更高的准确率。在解冻卷积层之后的训练中，我们需要使用非常小的学习率来保证不大幅度的改变卷积层。